<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/fastcampus/LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install --upgrade transformers

In [2]:
from datasets import load_dataset, DatasetDict

raw_dataset = load_dataset('ccdv/cnn_dailymail', '3.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
raw_dataset['train'][0]['article'][:200]

"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte"

In [5]:
raw_dataset['train'].to_pandas()

,article,highlights,id
0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,00027e965c8264c35cc1bc55556db388da82b07f
3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,0002c17436637c4fe1837c935c04de47adb18e9a
4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,0003ad6ef0c37534f80b55b4235108024b407f0b
...,...,...,...
287108,Tiger Woods’s frustration at the lamentable st...,"Woods said: ’Guys, give me a little f***ing sp...",fffdfb56fdf1a12d364562cc2b9b1d4de7481dee
287109,By . Mark Duell . Last updated at 4:07 PM on 2...,13 sailors died in 1804 after explosives ship ...,fffeecb8690b85de8c3faed80adbc7a978f9ae2a
287110,"Suicide: Troll victim Hannah Smith, 14, killed...",Hannah Smith's father says Ask.fm's safety cha...,ffff5231e4c71544bc6c97015cdb16c60e42b3f4
287111,By . Victoria Woollaston and Mark Prigg . PUBL...,A test version of Windows 8.1 is available to ...,ffff924b14a8d82058b6c1c5368ff1113c1632af


In [6]:
sampled_dataset = DatasetDict(
    {
        'train': raw_dataset['train'].select(range(50000)).shuffle(),
        'validation': raw_dataset['validation'].select(range(5000)).shuffle(),
        # 'test': raw_dataset['test'].select(range(5000))
    }
)

Tokenizer

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [8]:
# article을 대상으로 tokenizer train??

def get_training_corpus(ds):
    return(
        ds[i:i+1000]['article'] for i in range(0, len(ds), 1000)
    )

training_corpus = get_training_corpus(raw_dataset['train'])

In [9]:
%%time

tokenizer = tokenizer.train_new_from_iterator(training_corpus, vocab_size=50257)


CPU times: user 11min 52s, sys: 12.8 s, total: 12min 5s
Wall time: 1min 14s


In [10]:
sample_text = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte"

tokenizer.tokenize(sample_text)

['It',
 "'s",
 'Ġofficial',
 ':',
 'ĠU',
 '.',
 'S',
 '.',
 'ĠPresident',
 'ĠBarack',
 'ĠObama',
 'Ġwants',
 'Ġlawmakers',
 'Ġto',
 'Ġweigh',
 'Ġin',
 'Ġon',
 'Ġwhether',
 'Ġto',
 'Ġuse',
 'Ġmilitary',
 'Ġforce',
 'Ġin',
 'ĠSyria',
 '.',
 'ĠObama',
 'Ġsent',
 'Ġa',
 'Ġletter',
 'Ġto',
 'Ġthe',
 'Ġheads',
 'Ġof',
 'Ġthe',
 'ĠHouse',
 'Ġand',
 'ĠSenate',
 'Ġon',
 'ĠSaturday',
 'Ġnight',
 ',',
 'Ġhours',
 'Ġaf',
 'te']

In [11]:
tokenizer(sample_text)

{'input_ids': [868, 345, 1061, 26, 458, 14, 51, 14, 1497, 4149, 1288, 2880, 8505, 280, 6284, 285, 316, 1714, 280, 1321, 1681, 2692, 285, 2730, 14, 1288, 1170, 259, 3102, 280, 262, 6017, 290, 262, 1821, 292, 3897, 316, 1722, 1074, 12, 1511, 454, 865], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer(sample_text, return_length=True)

{'input_ids': [868, 345, 1061, 26, 458, 14, 51, 14, 1497, 4149, 1288, 2880, 8505, 280, 6284, 285, 316, 1714, 280, 1321, 1681, 2692, 285, 2730, 14, 1288, 1170, 259, 3102, 280, 262, 6017, 290, 262, 1821, 292, 3897, 316, 1722, 1074, 12, 1511, 454, 865], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'length': [44]}

In [13]:
context_length = 128

def tokenize(batch):
    outputs = tokenizer(
        batch['article'],
        max_length=context_length,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch = []
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length == context_length:
            input_batch.append(input_ids)
    return {'input_ids': input_batch}

In [14]:
tokenized_datasets = sampled_dataset.map(tokenize, batched=True, remove_columns=raw_dataset['train'].column_names)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Load model

In [15]:
from transformers import LlamaConfig

configuration = LlamaConfig()

configuration

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [16]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.vocab_size


(0, 0, 50257)

In [17]:
configuration = LlamaConfig(**{
  "attention_bias": False,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1376,
  "max_position_embeddings": 128,
  "mlp_bias": False,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": None,
  "rope_theta": 10000.0,
  "tie_word_embeddings": False,
  "transformers_version": "4.42.4",
  "use_cache": True,
  "vocab_size": 50257
})

In [18]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM(configuration)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((512,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((512,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((

In [19]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [20]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((512,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((512,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((

In [21]:
from operator import ge
prompt = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte"

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=50)
generate_ids

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tensor([[  868,   345,  1061,    26,   458,    14,    51,    14,  1497,  4149,
          1288,  2880,  8505,   280,  6284,   285,   316,  1714,   280,  1321,
          1681,  2692,   285,  2730,    14,  1288,  1170,   259,  3102,   280,
           262,  6017,   290,   262,  1821,   292,  3897,   316,  1722,  1074,
            12,  1511,   454,   865, 25882, 25882, 25882, 25882, 25882, 25882]],
       device='cuda:0')

In [22]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte vigorously vigorously vigorously vigorously vigorously vigorously"

Train model

In [24]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collactor = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [25]:
out = data_collactor([tokenized_datasets['train'][i] for i in range(3)])

for key in out:
    print(f"{key}: {out[key].shape}")

input_ids: torch.Size([3, 128])
attention_mask: torch.Size([3, 128])
labels: torch.Size([3, 128])


In [26]:
out['input_ids'][0][:20], out['attention_mask'][0][:20], out['labels'][0][:20]

(tensor([    8,  1356,     9,   630, 20301,   290,   691,   996,   439,  1360,
           285,   262, 17875,    12,  5840,    12, 10932,  2809,   669,   824]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([    8,  1356,     9,   630, 20301,   290,   691,   996,   439,  1360,
           285,   262, 17875,    12,  5840,    12, 10932,  2809,   669,   824]))

In [27]:
from transformers import TrainingArguments

batch_size = 32
logging_steps = 1000
learning_rate = 5e-4
num_epochs = 1

args = TrainingArguments(
    # output_dir='./results',
    output_dir='newsllama',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='steps',
    eval_steps=logging_steps,
    logging_steps=logging_steps,
    save_steps=logging_steps,
    gradient_accumulation_steps=8,
    num_train_epochs=num_epochs,
    weight_decay=0.1,
    warmup_steps=logging_steps,
    lr_scheduler_type='cosine',
    learning_rate=learning_rate,
    # save_total_limit=3,
    fp16=True,
    push_to_hub=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collactor,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [30]:
trainer.train()

Step,Training Loss,Validation Loss
1000,6.114200,5.358841


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=1139, training_loss=5.934852928315682, metrics={'train_runtime': 1019.0287, 'train_samples_per_second': 286.357, 'train_steps_per_second': 1.118, 'total_flos': 8595722395975680.0, 'train_loss': 5.934852928315682, 'epoch': 0.9992323719706108})

In [32]:
prompt = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours"
inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after the Senate passed a bill that would"

In [33]:
prompt = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in" # Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours"
inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in the U.S. military. The U.S. military is not the only U.S. military to be in the United"

In [35]:
prompt = """Shall I compare thee to a summer's day?
Thou art more lovely and more temperate:
Rough winds do shake the
"""
inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Shall I compare thee to a summer's day?\nThou art more lovely and more temperate:\nRough winds do shake the\n of the city's most famous city. The city's most famous city, the city of the"

In [34]:
prompt = "As a scientific endeavor, machine learning grew out of the quest for artificial intelligence. in the"
inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'As a scientific endeavor, machine learning grew out of the quest for artificial intelligence. in the United States, the United States and the United States have been working with the United States to help the United States and other countries. The United States has been working'

In [ ]:
model.save_pretrained('daily_llama')
tokenizer.save_pretrained('daily_tokenizer')